MERRA-2, from NASA, is an atmostpheric reanalysis that begins in 1980, but for wind plant energy analysis is usually only considered from a historic perspective from 1997-2000 due to significant changes in input data in the mid 1990's. See https://gmao.gsfc.nasa.gov/pubs/docs/Bosilovich785.pdf for more detail. The MERRA-2 data is comprised of a family of datasets, of which M2T1NXSLV hourly average and M2TMNXSLV monthly average are of the most interest for building wind plant power models. See pdf page 46, doc page 52 in same reference for a list of available variables included in the reanalysis.

Data can be accessed via web interface using the simple subsetting wizard (ssw) or https://disc.gsfc.nasa.gov/SSW/ or programmatically via an API https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Use%20the%20Web%20Services%20API%20for%20Subsetting%20MERRA-2%20Data or directly with wget and prepared scripts https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Download%20Hourly%20MERRA-2%20Time%20Series%20at%20a%20Single%20Point.

The notebook focuses on the programmatic approach, using python to pull data using the API. The main input is the latitude and longitude of the wind plant, in decimal degrees, with latitude <0 south of the equator, and longitude <0.

We begin by importing the required packages:

In [1]:
# if any of the packages below are not installed, uncomment
# line below and change packagename to package to be installed
# assumes you have anaconda installed

# !conda install --yes --prefix {sys.prefix} packagename

In [2]:
import sys
import os
import getpass
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import bisect
import numpy as np
import xarray as xr
import pandas as pd
from datetime import timedelta
import ipywidgets as widgets


For this code to work, you need a login at the NASA site.  Get a login here: https://urs.earthdata.nasa.gov/oauth/authorize?response_type=code&redirect_uri=https%3A%2F%2Fdisc.gsfc.nasa.gov%2Flogin%2Fcallback&client_id=C_kKX7TXHiCUqzt352ZwTQ 

After you have created an account, you must authorize that account for downloading data. See https://disc.gsfc.nasa.gov/earthdata-login. The update/approval is instantaneous, but download code below will not work until you do this step.  

In [3]:
# force a pause
input('did you authorize account?')

did you authorize account? y


'y'

To download files to work properly, you must have a $HOME/.netrc file 
that contains the following text (configured with your own Earthdata userid and password): 
    machine urs.earthdata.nasa.gov login [userid] password [password]

We will check to see if you have this file yet, and if not, prompt you for your username and password created at the link above.

In [4]:
hm=os.path.expanduser('~')
fpathname=hm+"/.netrc"

def createfile(fpathname) :
    username = input("Enter username: ")
    print('Enter password:')
    password = getpass.getpass()
    filetxt="machine urs.earthdata.nasa.gov login "+username+" password "+password
    f=open(fpathname,"a+")
    f.write(filetxt)
    f.close
    print("created/added to .netrc")
    
if os.path.exists(fpathname):
    print("you have a .netrc file")
    f=open(fpathname)
    for line in f:
        #print(line)
        if 'urs.earthdata.nasa.gov' in line:
            print('it has urs.earthdata.nasa.gov')
            break
        else:
            print("missing urs.earthdata.nasa.gov")
            createfile(fpathname)    
else:
    print(".netrc does not exist, creating one...")
    createfile(fpathname) 

    

you have a .netrc file
it has urs.earthdata.nasa.gov


The MERRA-2 data is for a matrix of datapoints spaced at 0.625 degrees intervals of longitude and 0.5% of latitude for a grid of 576 points longitude and 361 points latitude.

In [5]:
x=np.arange(-180+0.625,180+0.625,0.625) # longitude -180,180 overlap so only one counted
print('points of longitude: ' + str(len(x)))
print(x[[0,1,2,573,574,575]])
y=np.arange(-90,90.5,0.5) # latitude 
print('points of latitude: ' + str(len(y)))
print(y[[0,1,2,358,359,360]])

points of longitude: 576
[-179.375 -178.75  -178.125  178.75   179.375  180.   ]
points of latitude: 361
[-90.  -89.5 -89.   89.   89.5  90. ]


In almost all cases, the location we're interested in will not be located exactly at one of the MERRA-2 data grid points, so we build a bounding box around the point and get the data for the 4 surrounding points and interpolate between these 4 points to approximate site conditions.

In [6]:
# this is the location for La Haute Borne 4 turbine wind plant in France
# This is the example in the NREL OpenOA notebooks, and KEI notebooks.
input_lat = 48.451194
input_lon = 5.589603

def bounding_box(lat,lon):
    #bisect only works with positive numbers so shift all positive
    #then move back after indexes identified
    x = np.linspace(-180.0,180.0,577)
    y = np.linspace(-90.0,90.0,361)
    #bisect doesn't work with negative numbers, so add 90/180 to make range positive
    ind_y = bisect.bisect_left(y+90,lat+90)
    ind_x = bisect.bisect_left(x+180,lon+180)

    return [x[ind_x-1],y[ind_y-1],x[ind_x],y[ind_y]]

bbox=bounding_box(input_lat,input_lon)
print(bbox)


[5.0, 48.0, 5.625, 48.5]


Now that we know where, we need to know when. This code only downloads 5 days of data, as it is time consuming. It is common to retrieve historic data back to Jan 1 1997 or Jan 1 2000. We also need data concurrent with the operational period of the wind plant to be studied, so we usually get data up through the present, usually 2 months behind the current date. As of 22 Oct 2020, data for MERRA-2 is available through the end of August 2020. There is usually a 4-6 week lag as the reanalysis data is prepared from observational data.

Note that the data is stored daily, so it is also possible to subset specific time ranges within each day, as well as selecting specific date ranges.

In [7]:
#begTime = '2010-01-01'
begTime = '2014-01-01'
#endTime = '2020-09-01'   # specifying 6 Jan gets data through 5 Jan.
endTime = '2016-01-01'   # specifying 6 Jan gets data through 5 Jan.
begHour = '00:30'
endHour = '23:30'


Finally, we must specify the dataset to pull from, and within that dataset, specifically which variables in which we are interested. See the first link in the introductory paragraph, pdf page 46, doc page 52 for the list of available variables and descriptions.

In [8]:
product = 'M2T1NXSLV_5.12.4'
varNames =['TS','T2MDEW','T2M', 'T10M', 'T850',
           'H1000', 'H850','SLP', 'PS','PBLTOP',
           'U2M','V2M','U10M','V10M','U50M','V50M','U850','V850']
print(product)
print(varNames)

M2T1NXSLV_5.12.4
['TS', 'T2MDEW', 'T2M', 'T10M', 'T850', 'H1000', 'H850', 'SLP', 'PS', 'PBLTOP', 'U2M', 'V2M', 'U10M', 'V10M', 'U50M', 'V50M', 'U850', 'V850']


Time to get the data.  We'll pass this off to a series of functions from the NASA site. The desired spatial and temporal constraints, along with the dataset and variable 
specifications, are stored in a JSON-based Web Service Protocol (WSP) structure, 
which is named “subset_request”. 


In [9]:
subset_request = {
    'methodname': 'subset',
    'type': 'jsonwsp/request',
    'version': '1.0',
    'args': {
        'role'  : 'subset',
        'start' : begTime,
        'end'   : endTime,
        'box'   : bbox,
        'crop'  : True, 
        'diurnalFrom': begHour, 
        'diurnalTo': endHour,
        'diurnalMean': True,
        'data': [{'datasetId': product,'variable' : varNames[0]},
                 {'datasetId': product,'variable' : varNames[1]},
                 {'datasetId': product,'variable' : varNames[2]},
                 {'datasetId': product,'variable' : varNames[3]},
                 {'datasetId': product,'variable' : varNames[4]},
                 {'datasetId': product,'variable' : varNames[5]},
                 {'datasetId': product,'variable' : varNames[6]},
                 {'datasetId': product,'variable' : varNames[7]},
                 {'datasetId': product,'variable' : varNames[8]},
                 {'datasetId': product,'variable' : varNames[9]},
                 {'datasetId': product,'variable' : varNames[10]},
                 {'datasetId': product,'variable' : varNames[11]},
                 {'datasetId': product,'variable' : varNames[12]},
                 {'datasetId': product,'variable' : varNames[13]},
                 {'datasetId': product,'variable' : varNames[14]},
                 {'datasetId': product,'variable' : varNames[15]}]
    }
}

the JSON-formatted subset_request is POSTed to the GES DISC server. The Job ID is extracted from the response and will be used later as a reference for the request.

In [10]:
# Create a urllib PoolManager instance to make requests.
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
# Set the URL for the GES DISC subset service endpoint
url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'

def get_http_data(request,url):
    hdrs = {'Content-Type': 'application/json',
            'Accept'      : 'application/json'}
    data = json.dumps(request)       
    r = http.request('POST', url, body=data, headers=hdrs)
    response = json.loads(r.data)   
    # Check for errors
    if response['type'] == 'jsonwsp/fault' :
        print('API Error: faulty %s request' % response['methodname'])
        sys.exit(1)
    return response

# Submit the subset request to the GES DISC Server
response = get_http_data(subset_request,url)
# Report the JobID and initial status
myJobId = response['result']['jobId']
print('Job ID: '+myJobId)
print('Job status: '+response['result']['Status'])

Job ID: 5f944f476980f76c0d0e358e
Job status: Accepted


At this point, the job is running on the GES DISC server. The next step is to construct another JSON WSP status_request, with methodname parameter set to 'GetStatus'. The args parameter contains the extracted Job ID. The status_request is submitted periodically to monitor the job status as it changes from 'Accepted' to 'Running' to '100% completed'. When the job is finished, check on the final status to ensure the job succeeded.


In [11]:
# Construct JSON WSP request for API method: GetStatus
status_request = {
    'methodname': 'GetStatus',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {'jobId': myJobId}
}

# Check on the job status after a brief nap
while response['result']['Status'] in ['Accepted', 'Running']:
    sleep(5)
    response = get_http_data(status_request,url)
    status  = response['result']['Status']
    percent = response['result']['PercentCompleted']
    print ('Job status: %s (%d%c complete)' % (status,percent,'%'))
if response['result']['Status'] == 'Succeeded' :
    print ('Job Finished:  %s' % response['result']['message'])
else : 
    print('Job Failed: %s' % response['fault']['code'])
    sys.exit(1)

Job status: Succeeded (100% complete)
Job Finished:  Complete (M2T1NXSLV_5.12.4)


In [12]:
# Construct JSON WSP request for API method: GetResult
batchsize = 20
results_request = {
    'methodname': 'GetResult',
    'version': '1.0',
    'type': 'jsonwsp/request',
    'args': {
        'jobId': myJobId,
        'count': batchsize,
        'startIndex': 0
    }
}

# Retrieve the results in JSON in multiple batches 
# Initialize variables, then submit the first GetResults request
# Add the results from this batch to the list and increment the count
results = []
count = 0 
response = get_http_data(results_request,url) 
count = count + response['result']['itemsPerPage']
results.extend(response['result']['items']) 

# Increment the startIndex and keep asking for more results until we have them all
total = response['result']['totalResults']
while count < total :
    results_request['args']['startIndex'] += batchsize 
    response = get_http_data(results_request,url) 
    count = count + response['result']['itemsPerPage']
    results.extend(response['result']['items'])
       
# Check on the bookkeeping
print('Retrieved %d out of %d expected items' % (len(results), total))

Retrieved 732 out of 732 expected items


Sort the results into documents and URLs

In [13]:
docs = []
urls = []
for item in results :
    try:
        if item['start'] and item['end'] : urls.append(item) 
    except:
        docs.append(item)
# Print out the documentation links, but do not download them
print('\nDocumentation:')
for item in docs : print('\n'+item['label']+': '+item['link'])


Documentation:

README Document: https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2T1NXSLV.5.12.4/doc/MERRA2.README.pdf


In [14]:
# if getting a large date range comment out these lines of code with # sign
print('\n first 2 data URLS:')
for item in urls[0:2] : print('\n'+item['label']+': '+item['link'])



 first 2 data URLS:

MERRA2_400.tavg1_2d_slv_Nx.20140101.SUB.nc: https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXSLV.5.12.4%2F2014%2F01%2FMERRA2_400.tavg1_2d_slv_Nx.20140101.nc4&FORMAT=bmM0Lw&BBOX=48%2C5%2C48.5%2C5.625&TIME=2014-01-01T00%3A30%3A00%2F2014-01-01T23%3A30%3A59&LABEL=MERRA2_400.tavg1_2d_slv_Nx.20140101.SUB.nc&SHORTNAME=M2T1NXSLV&SERVICE=L34RS_MERRA2&VERSION=1.02&DATASET_VERSION=5.12.4&VARIABLES=TS%2CT2MDEW%2CT2M%2CT10M%2CT850%2CH1000%2CH850%2CSLP%2CPS%2CPBLTOP%2CU2M%2CV2M%2CU10M%2CV10M%2CU50M%2CV50M

MERRA2_400.tavg1_2d_slv_Nx.20140102.SUB.nc: https://goldsmr4.gesdisc.eosdis.nasa.gov/daac-bin/OTF/HTTP_services.cgi?FILENAME=%2Fdata%2FMERRA2%2FM2T1NXSLV.5.12.4%2F2014%2F01%2FMERRA2_400.tavg1_2d_slv_Nx.20140102.nc4&FORMAT=bmM0Lw&BBOX=48%2C5%2C48.5%2C5.625&TIME=2014-01-02T00%3A30%3A00%2F2014-01-02T23%3A30%3A59&LABEL=MERRA2_400.tavg1_2d_slv_Nx.20140102.SUB.nc&SHORTNAME=M2T1NXSLV&SERVICE=L34RS_MERRA2&VERSION=1.02&DATASET_V

Get a login here: https://urs.earthdata.nasa.gov/oauth/authorize?response_type=code&redirect_uri=https%3A%2F%2Fdisc.gsfc.nasa.gov%2Flogin%2Fcallback&client_id=C_kKX7TXHiCUqzt352ZwTQ 

Use the requests library to submit the HTTP_Services URLs anddownload the files. Note that they are being posted to a downloads folder within the current folder. 


In [15]:
download_folder='downloads'
try:
    os.mkdir(download_folder)
    print("Directory '% s' created" % download_folder)
except OSError as error:  
    print(error) 

[Errno 17] File exists: 'downloads'


There is one file per day.  It takes 10-20 seconds per file, so 5 files can take a minute or more to download. This translates roughly to 5-10 minutes per month, 1-2 hours per year, so a 20 year dataset can take more than 24 hours to download.

As noted above when specifying end date, you have to go one past the date you want. the last item does not download for some reason. We drop the last item from urls to prevent getting errors when downloading.

In [16]:
# this is a bandaid fix because code always broke on last URL, not sure why
urls = urls[0:len(urls)-1]

In [17]:
num_files=len(urls)
est_min=round((num_files*18)/60,1)
est_hrs=round(est_min/60,3)

In [18]:
print('This is the long part. you are downloading '+str(num_files)+' files.')
print('This will take about '+str(est_min)+' minutes, or '+str(est_hrs)+' hours.' )
print('The name of each file will be printed as it downloads.')
response=input('Do you want to continue (y/n): ')
if response in ['n','no','N','No','NO','nyet']:
    raise SystemExit("Stop right there!")

out=widgets.Output()
pb=widgets.IntProgress( value=0, min=0, max=num_files, 
                           step=1,description='Progress:', 
                           bar_style='success',orientation='horizontal')
display(pb)
print('\nHTTP_services output:')

for item in urls :

    URL = item['link'] 
    result = requests.get(URL)
    try:
        result.raise_for_status()
        outfn = download_folder+'/'+item['label']
        f = open(outfn,'wb')
        f.write(result.content)
        f.close()
        print(outfn)
        pb.value +=1
    except:
        print('Error! Status code is %d for this URL:\n%s' % (result.status_code,URL))
        print('Help for downloading data is at https://disc.gsfc.nasa.gov/data-access')
print('OK')

This is the long part. you are downloading 730 files.
This will take about 219.0 minutes, or 3.65 hours.
The name of each file will be printed as it downloads.


Do you want to continue (y/n):  y


IntProgress(value=0, bar_style='success', description='Progress:', max=730)


HTTP_services output:
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140101.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140102.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140103.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140104.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140105.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140106.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140107.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140108.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140109.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140110.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140111.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140112.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140113.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140114.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140115.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140116.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140117.SUB.nc
downloads/MERRA2_400.tavg1_2d_slv_Nx.20140118.SUB.nc
downloads/MERRA2_400.ta

When downloading multiple years of files, it makes sense to check the files downloaded vs the target and also the file size. Files for the list of variables above are generally around 122 kB in size and we'll check that first.

In [19]:
file_list = os.listdir(download_folder)
for f in file_list:
    fpath=download_folder+"/"+f
    print(f+' : ' + str(round(os.path.getsize(fpath)/1000))+' kB')


MERRA2_400.tavg1_2d_slv_Nx.20150630.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150922.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20141201.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20141031.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20151230.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150905.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20140620.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150911.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20140820.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150816.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150627.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150202.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20140306.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20151007.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20141126.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20140622.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20140208.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20151105.SUB.nc : 122 kB
MERRA2_400.tavg1_2d_slv_Nx.20150524.SUB.nc : 122 kB
MERRA2_400.t

Now we'll cycle through the original intended download list and make sure there is a file in {download_folder}

In [20]:
for item in urls :
    path=download_folder+"/"+item['label']
    print(item['label'] + ", downloaded =  " + str(os.path.isfile(path)))

MERRA2_400.tavg1_2d_slv_Nx.20140101.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140102.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140103.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140104.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140105.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140106.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140107.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140108.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140109.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140110.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140111.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140112.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140113.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140114.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140115.SUB.nc, downloaded =  True
MERRA2_400.tavg1_2d_slv_Nx.20140116.SUB.nc, downloaded 

Now that download is complete and we've verified we got the files we wanted, it's time to mine the data from the files. The files are in netCDF format. There are several python packages for working with netCDF format, but xarray offers a simple conversion to pandas dataframe.

We will read in and combine the data and save as csv to an Output/raw_data folder as the final step in this notebook. processing the data will be done in the next notebook.

In [21]:
mainout_folder='Output'
try:
    os.mkdir(mainout_folder)
    print("Directory '% s' created" % mainout_folder)
except OSError as error:  
    print(error) 

rawout_folder='raw_data'
try:
    os.mkdir(mainout_folder+'/'+rawout_folder)
    print("Directory '% s' created" % rawout_folder)
except OSError as error:  
    print(error) 

out_path=mainout_folder+'/'+rawout_folder
out_path

[Errno 17] File exists: 'Output'
[Errno 17] File exists: 'Output/raw_data'


'Output/raw_data'

In [22]:
raw_data=[]
for f in file_list:
    fpath=download_folder+'/'+f
    ds = xr.open_dataset(fpath)
    df = ds.to_dataframe()
    raw_data.append(df)
print('\nfiles imported into list: ',str(len(raw_data)) )
raw_data = pd.concat(raw_data)

print('\nshape (rows and columns): ' + str(raw_data.shape))
print('\ncolumn names: ' + str(raw_data.columns))
print('\n\n')
print(raw_data.describe(include='all'))
print('\n\n')
print(raw_data.head())


files imported into list:  730

shape (rows and columns): (70080, 16)

column names: Index(['H1000', 'H850', 'PBLTOP', 'PS', 'SLP', 'T10M', 'T2M', 'T2MDEW', 'T850',
       'TS', 'U10M', 'U2M', 'U50M', 'V10M', 'V2M', 'V50M'],
      dtype='object')



              H1000          H850         PBLTOP             PS  \
count  70080.000000  70080.000000   70080.000000   70080.000000   
mean     141.747406   1489.164185   90942.164062   98049.898438   
std       65.731392     70.993355    6519.765625     937.399109   
min     -199.625656   1100.897461   58707.269531   93514.632812   
25%      103.807066   1449.178375   85949.156250   97458.242188   
50%      143.790886   1499.828918   92673.890625   98010.835938   
75%      183.447575   1539.109009   96717.892578   98672.642578   
max      326.205658   1639.071899  100355.695312  101136.867188   

                 SLP          T10M           T2M        T2MDEW          T850  \
count   70080.000000  70080.000000  70080.000000  70080.000000  7

And finally we can write the combined raw data dataframe to csv format. Here we save 'raw_data.csv' out_path folder created above

In [23]:
out_file='raw_data.csv'
full_out_path=out_path+'/'+out_file
raw_data.to_csv(full_out_path)

print(os.listdir(out_path))

['raw_data.csv']


In [24]:
read_in_raw=pd.read_csv(full_out_path)
read_in_raw['time']=pd.to_datetime(read_in_raw['time'], format='%Y-%m-%d %H:%M:%S')

print(read_in_raw.dtypes)
print('\n\n')
print(read_in_raw.describe(include='all'))
print('\n\n')
print(read_in_raw)
print('\n\n')

lat              float64
lon              float64
time      datetime64[ns]
H1000            float64
H850             float64
PBLTOP           float64
PS               float64
SLP              float64
T10M             float64
T2M              float64
T2MDEW           float64
T850             float64
TS               float64
U10M             float64
U2M              float64
U50M             float64
V10M             float64
V2M              float64
V50M             float64
dtype: object



                 lat           lon                 time         H1000  \
count   70080.000000  70080.000000                70080  70080.000000   
unique           NaN           NaN                17520           NaN   
top              NaN           NaN  2015-01-06 18:30:00           NaN   
freq             NaN           NaN                    4           NaN   
first            NaN           NaN  2014-01-01 00:30:00           NaN   
last             NaN           NaN  2015-12-31 23:30:00           NaN 

in the specifications for the date range above, we set the start and end date begTime and endTime. we should make sure we got 4 datapoints for each - one for each the surrounding bounding box locations from our initial lat/lon.

In [25]:
begTime=pd.to_datetime(begTime,format='%Y-%m-%d')
begTime=begTime+timedelta(minutes=30)
print(begTime)
endTime=pd.to_datetime(endTime,format='%Y-%m-%d')
endTime=endTime-timedelta(minutes=30) # need to drop an hour
print(endTime)
expected_timestamps= pd.date_range(start=begTime,end=endTime,freq='H')
print(type(expected_timestamps))
print(expected_timestamps)
actual_timestamps=pd.to_datetime(read_in_raw['time'].unique(),format='%Y-%d-%mT%H:%M:%S').sort_values()
print(type(actual_timestamps))
print(actual_timestamps)

print('\n\nDo they all match?  '+str(all(expected_timestamps==actual_timestamps)))

2014-01-01 00:30:00
2015-12-31 23:30:00
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2014-01-01 00:30:00', '2014-01-01 01:30:00',
               '2014-01-01 02:30:00', '2014-01-01 03:30:00',
               '2014-01-01 04:30:00', '2014-01-01 05:30:00',
               '2014-01-01 06:30:00', '2014-01-01 07:30:00',
               '2014-01-01 08:30:00', '2014-01-01 09:30:00',
               ...
               '2015-12-31 14:30:00', '2015-12-31 15:30:00',
               '2015-12-31 16:30:00', '2015-12-31 17:30:00',
               '2015-12-31 18:30:00', '2015-12-31 19:30:00',
               '2015-12-31 20:30:00', '2015-12-31 21:30:00',
               '2015-12-31 22:30:00', '2015-12-31 23:30:00'],
              dtype='datetime64[ns]', length=17520, freq='H')
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>
DatetimeIndex(['2014-01-01 00:30:00', '2014-01-01 01:30:00',
               '2014-01-01 02:30:00', '2014-01-01 03:30:00',
               '2014-01-01 04:30:00', 

In [ ]:
input('the end...')